# Accuracy experiments with Infection Benchmark

## Load and visualize the data

In [57]:
# Import the utility functions
from attention_analysis_utils import (
    get_attention_raw_dict,
    process_attention_dict,
    get_computation_graph,
    get_nodes_per_level_from_comp_graph_full,
    get_attention_raw_dict_multihead,
    reindex_nodes_per_level,
    translate_comp_graph,
    get_att_dict_per_layer,
    return_edges_in_k_hop,
    get_ATTATTTRIBUTE_edge,
    get_AVGATT_edge,
    average_attention_heads,
)

from torch_geometric.utils import get_num_hops
from visualization_utils import (
    visualize_computation_graph,
)
import torch
from typing import Tuple

def get_edge_scores(
    target_edge: Tuple, comp_graph, comp_graph_new, layer_att_dict, att
):
    assert type(target_edge) == tuple, "target_edge must be a tuple"
    # Get ATTATTRIBUTE & ATTATTRIBUTE_sim scores
    attattribute, attattribute_sim = get_ATTATTTRIBUTE_edge(
        comp_graph=comp_graph,
        comp_graph_new=comp_graph_new,
        layer_att_dict=layer_att_dict,
        target_edge=target_edge,
        verbose=False,
    )
    # Get AVGATT scores
    avgatt = get_AVGATT_edge(att=att, edge=target_edge)

    return attattribute, attattribute_sim, avgatt

def return_is_edge_list_Infection(edge_list, path_expl):
    # Assuming path_expl is something like:
    # [1215, 1024, 606, 10]. We need all edges in edge_list
    # to be checked for the presence of this path.
    expl_edge_set = {(path_expl[i], path_expl[i+1]) for i in range(len(path_expl) - 1)}

    ground_truth_edge_list = []
    for edge in edge_list:
        if tuple(edge) in expl_edge_set:
            ground_truth_edge_list.append(1)
        else:
            ground_truth_edge_list.append(0)
    return ground_truth_edge_list

def experiment_on_target_node(
    target_idx: int, data, model, path_expl, self_loops=True, multiheads=False,
):
    num_hops = get_num_hops(model)
    num_layers = num_hops

    edge_lists = return_edges_in_k_hop(
        data=data, target_idx=target_idx, hop=2, self_loops=self_loops
    )
    # 3. For all edges in the k-hop neighborhood, we get the attribution scores
    # according to ATTATTRIBUTE, ATTATTRIBUTE_sim, and AVGATT.
    # First, prepare ingredients for analysis

    num_layers = get_num_hops(model)
    if multiheads:
        att_dict_raw = get_attention_raw_dict_multihead(model, data)
    else:
        att_dict_raw = get_attention_raw_dict(model, data)
    att_dict = process_attention_dict(att_dict_raw)
    comp_graph = get_computation_graph(
        edge_index=data.edge_index, k=num_layers, target_idx=target_idx
    )
    (
        nodes_per_level_original,
        num_nodes_per_level,
        true_node_label,
    ) = get_nodes_per_level_from_comp_graph_full(comp_graph=comp_graph)
    nodes_per_level_new = reindex_nodes_per_level(
        nodes_per_level_original, num_nodes_per_level
    )
    comp_graph_new = translate_comp_graph(
        comp_graph=comp_graph,
        nodes_per_level_new=nodes_per_level_new,
        nodes_per_level_original=nodes_per_level_original,
    )
    layer_att_dict = get_att_dict_per_layer(
        comp_graph=comp_graph, comp_graph_new=comp_graph_new, att_dict=att_dict
    )

    # Get results for all edges in the k-hop neighborhood
    attattribute_list, attattribute_sim_list, avgatt_list = [], [], []
    # Get the attention weights again
    with torch.no_grad():
        model(data.x, data.edge_index, return_att=True)
        att = model.att 
        att = average_attention_heads(att)
        model.att = att

    for current_edge in edge_lists:
        attattribute, attattribute_sim, avgatt = get_edge_scores(
            target_edge=tuple(current_edge),
            comp_graph=comp_graph,
            comp_graph_new=comp_graph_new,
            layer_att_dict=layer_att_dict,
            att=att,
        )
        attattribute_list.append(attattribute)
        attattribute_sim_list.append(attattribute_sim)
        avgatt_list.append(avgatt)

    ground_truth_edge_list = return_is_edge_list_Infection(edge_lists, path_expl)

    return (
        attattribute_list,
        attattribute_sim_list,
        avgatt_list,
        ground_truth_edge_list,
    )

GAT 3 layer 1 head

In [1]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx, k_hop_subgraph, remove_self_loops, add_self_loops
from torch_geometric.data import Data

dataset_name = 'Infection_50003d_sp'
model_name = f'GAT_infection_3L1H_sp'

# Load the data
data = torch.load(f'/workspace/{dataset_name}.pt',map_location ='cpu')
# Load the model
model = torch.load(f'/workspace/{model_name}.pt',map_location ='cpu')
model.eval()
# Get the attention weights
with torch.no_grad():
    out = model(data.x, data.edge_index, return_att=True)
    att = model.att 
    # att = average_attention_heads(att)
    # model.att = att

# data.edge_index = add_self_loops(remove_self_loops(data.edge_index)[0])[0]
# G = to_networkx(data, to_undirected=True)   

In [59]:
attattribute_list, attattribute_sim_list, avgatt_list = [], [], []
ground_truth_edge_list = []

for idx, target_node in enumerate(data.unique_solution_nodes):
    path_expl = data.unique_solution_explanations[idx]
    target_node_results = experiment_on_target_node(
        target_idx=target_node,
        data=data,
        model=model,
        path_expl=path_expl,
        self_loops=True,
    )
    attattribute_list_curr = target_node_results[0]
    attattribute_sim_list_curr = target_node_results[1]
    avgatt_list_curr = target_node_results[2]
    ground_truth_edge_list_curr = target_node_results[3]

    attattribute_list.extend(attattribute_list_curr)
    attattribute_sim_list.extend(attattribute_sim_list_curr)
    avgatt_list.extend(avgatt_list_curr)
    ground_truth_edge_list.extend(ground_truth_edge_list_curr)

torch.save(
    torch.Tensor(attattribute_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(attattribute_sim_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(avgatt_list), f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt"
)
torch.save(
    torch.Tensor(ground_truth_edge_list),
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt",
)

In [60]:
# Load experiments from local

attattribute_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt"
)
attattribute_sim_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt"
)
avgatt_list = torch.load(f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt")
ground_truth_edge_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt"
)

In [61]:
from scipy.stats import kendalltau, spearmanr, pearsonr
from sklearn.metrics import roc_auc_score

ground_truth_edge_list = torch.Tensor(ground_truth_edge_list)
# Also include a random baseline
random_attr = torch.rand(ground_truth_edge_list.shape)
random_attr_roc_auc = roc_auc_score(ground_truth_edge_list, random_attr)
attattribute_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_list)
attattribute_sim_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_sim_list)
avgatt_roc_auc = roc_auc_score(ground_truth_edge_list, avgatt_list)

# Print results
print("ROC AUC for ATTATTRIBUTE / ATTATTRIBUTE_SIM / AVGATT / RANDOM")
print(f"{attattribute_roc_auc:.4f}, {attattribute_sim_roc_auc:.4f}, {avgatt_roc_auc:.4f}, {random_attr_roc_auc:.4f}")

ROC AUC for ATTATTRIBUTE / ATTATTRIBUTE_SIM / AVGATT / RANDOM
0.9359, 0.9405, 0.8852, 0.5073


GAT 3 layer 2 head

In [62]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx, k_hop_subgraph, remove_self_loops, add_self_loops
from torch_geometric.data import Data

dataset_name = 'Infection_50003d_sp'
model_name = f'GAT_infection_3L2H_sp'

# Load the data
data = torch.load(f'/workspace/{dataset_name}.pt',map_location ='cpu')
# Load the model
model = torch.load(f'/workspace/{model_name}.pt',map_location ='cpu')
model.eval()
# Get the attention weights
with torch.no_grad():
    out = model(data.x, data.edge_index, return_att=True)
    att = model.att 
    # att = average_attention_heads(att)
    # model.att = att

# data.edge_index = add_self_loops(remove_self_loops(data.edge_index)[0])[0]
# G = to_networkx(data, to_undirected=True)   

In [63]:
attattribute_list, attattribute_sim_list, avgatt_list = [], [], []
ground_truth_edge_list = []

for idx, target_node in enumerate(data.unique_solution_nodes):
    path_expl = data.unique_solution_explanations[idx]
    target_node_results = experiment_on_target_node(
        target_idx=target_node,
        data=data,
        model=model,
        path_expl=path_expl,
        self_loops=True,
        multiheads=True,
    )
    attattribute_list_curr = target_node_results[0]
    attattribute_sim_list_curr = target_node_results[1]
    avgatt_list_curr = target_node_results[2]
    ground_truth_edge_list_curr = target_node_results[3]

    attattribute_list.extend(attattribute_list_curr)
    attattribute_sim_list.extend(attattribute_sim_list_curr)
    avgatt_list.extend(avgatt_list_curr)
    ground_truth_edge_list.extend(ground_truth_edge_list_curr)

torch.save(
    torch.Tensor(attattribute_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(attattribute_sim_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(avgatt_list), f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt"
)
torch.save(
    torch.Tensor(ground_truth_edge_list),
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt",
)

In [64]:
# Load experiments from local

attattribute_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt"
)
attattribute_sim_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt"
)
avgatt_list = torch.load(f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt")
ground_truth_edge_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt"
)

In [65]:
from scipy.stats import kendalltau, spearmanr, pearsonr
from sklearn.metrics import roc_auc_score

ground_truth_edge_list = torch.Tensor(ground_truth_edge_list)
# Also include a random baseline
random_attr = torch.rand(ground_truth_edge_list.shape)
random_attr_roc_auc = roc_auc_score(ground_truth_edge_list, random_attr)
attattribute_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_list)
attattribute_sim_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_sim_list)
avgatt_roc_auc = roc_auc_score(ground_truth_edge_list, avgatt_list)

# Print results
# print("ROC AUC for ATTATTRIBUTE / ATTATTRIBUTE_SIM / AVGATT / RANDOM")
# print(f"{attattribute_roc_auc:.4f}, {attattribute_sim_roc_auc:.4f}, {avgatt_roc_auc:.4f}, {random_attr_roc_auc:.4f}")

GAT 3 layer 4 head

In [66]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx, k_hop_subgraph, remove_self_loops, add_self_loops
from torch_geometric.data import Data

dataset_name = 'Infection_50003d_sp'
model_name = f'GAT_infection_3L4H_sp'

# Load the data
data = torch.load(f'/workspace/{dataset_name}.pt',map_location ='cpu')
# Load the model
model = torch.load(f'/workspace/{model_name}.pt',map_location ='cpu')
model.eval()
# Get the attention weights
with torch.no_grad():
    out = model(data.x, data.edge_index, return_att=True)
    att = model.att 
    # att = average_attention_heads(att)
    # model.att = att

# data.edge_index = add_self_loops(remove_self_loops(data.edge_index)[0])[0]
# G = to_networkx(data, to_undirected=True)   

In [67]:
attattribute_list, attattribute_sim_list, avgatt_list = [], [], []
ground_truth_edge_list = []

for idx, target_node in enumerate(data.unique_solution_nodes):
    path_expl = data.unique_solution_explanations[idx]
    target_node_results = experiment_on_target_node(
        target_idx=target_node,
        data=data,
        model=model,
        path_expl=path_expl,
        self_loops=True,
        multiheads=True,
    )
    attattribute_list_curr = target_node_results[0]
    attattribute_sim_list_curr = target_node_results[1]
    avgatt_list_curr = target_node_results[2]
    ground_truth_edge_list_curr = target_node_results[3]

    attattribute_list.extend(attattribute_list_curr)
    attattribute_sim_list.extend(attattribute_sim_list_curr)
    avgatt_list.extend(avgatt_list_curr)
    ground_truth_edge_list.extend(ground_truth_edge_list_curr)

torch.save(
    torch.Tensor(attattribute_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(attattribute_sim_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(avgatt_list), f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt"
)
torch.save(
    torch.Tensor(ground_truth_edge_list),
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt",
)

In [68]:
# Load experiments from local

attattribute_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt"
)
attattribute_sim_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt"
)
avgatt_list = torch.load(f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt")
ground_truth_edge_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt"
)

In [69]:
from scipy.stats import kendalltau, spearmanr, pearsonr
from sklearn.metrics import roc_auc_score

ground_truth_edge_list = torch.Tensor(ground_truth_edge_list)
# Also include a random baseline
random_attr = torch.rand(ground_truth_edge_list.shape)
random_attr_roc_auc = roc_auc_score(ground_truth_edge_list, random_attr)
attattribute_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_list)
attattribute_sim_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_sim_list)
avgatt_roc_auc = roc_auc_score(ground_truth_edge_list, avgatt_list)

# Print results
# print("ROC AUC for ATTATTRIBUTE / ATTATTRIBUTE_SIM / AVGATT / RANDOM")
# print(f"{attattribute_roc_auc:.4f}, {attattribute_sim_roc_auc:.4f}, {avgatt_roc_auc:.4f}, {random_attr_roc_auc:.4f}")

GAT 3 layer 8 head

In [70]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx, k_hop_subgraph, remove_self_loops, add_self_loops
from torch_geometric.data import Data

dataset_name = 'Infection_50003d_sp'
model_name = f'GAT_infection_3L8H_sp'

# Load the data
data = torch.load(f'/workspace/{dataset_name}.pt',map_location ='cpu')
# Load the model
model = torch.load(f'/workspace/{model_name}.pt',map_location ='cpu')
model.eval()
# Get the attention weights
with torch.no_grad():
    out = model(data.x, data.edge_index, return_att=True)
    att = model.att 
    # att = average_attention_heads(att)
    # model.att = att

# data.edge_index = add_self_loops(remove_self_loops(data.edge_index)[0])[0]
# G = to_networkx(data, to_undirected=True)   

In [71]:
attattribute_list, attattribute_sim_list, avgatt_list = [], [], []
ground_truth_edge_list = []

for idx, target_node in enumerate(data.unique_solution_nodes):
    path_expl = data.unique_solution_explanations[idx]
    target_node_results = experiment_on_target_node(
        target_idx=target_node,
        data=data,
        model=model,
        path_expl=path_expl,
        self_loops=True,
        multiheads=True,
    )
    attattribute_list_curr = target_node_results[0]
    attattribute_sim_list_curr = target_node_results[1]
    avgatt_list_curr = target_node_results[2]
    ground_truth_edge_list_curr = target_node_results[3]

    attattribute_list.extend(attattribute_list_curr)
    attattribute_sim_list.extend(attattribute_sim_list_curr)
    avgatt_list.extend(avgatt_list_curr)
    ground_truth_edge_list.extend(ground_truth_edge_list_curr)

torch.save(
    torch.Tensor(attattribute_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(attattribute_sim_list),
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt",
)
torch.save(
    torch.Tensor(avgatt_list), f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt"
)
torch.save(
    torch.Tensor(ground_truth_edge_list),
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt",
)

In [72]:
# Load experiments from local

attattribute_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_list_Accuracy_test.pt"
)
attattribute_sim_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_attattribute_sim_list_Accuracy_test.pt"
)
avgatt_list = torch.load(f"/workspace/{dataset_name}_{model_name}_avgatt_list_Accuracy_test.pt")
ground_truth_edge_list = torch.load(
    f"/workspace/{dataset_name}_{model_name}_ground_truth_edge_list_Accuracy_test.pt"
)

In [73]:
from scipy.stats import kendalltau, spearmanr, pearsonr
from sklearn.metrics import roc_auc_score

ground_truth_edge_list = torch.Tensor(ground_truth_edge_list)
# Also include a random baseline
random_attr = torch.rand(ground_truth_edge_list.shape)
random_attr_roc_auc = roc_auc_score(ground_truth_edge_list, random_attr)
attattribute_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_list)
attattribute_sim_roc_auc = roc_auc_score(ground_truth_edge_list, attattribute_sim_list)
avgatt_roc_auc = roc_auc_score(ground_truth_edge_list, avgatt_list)

# Print results
print("ROC AUC for ATTATTRIBUTE / ATTATTRIBUTE_SIM / AVGATT / RANDOM")
print(f"{attattribute_roc_auc:.4f}, {attattribute_sim_roc_auc:.4f}, {avgatt_roc_auc:.4f}, {random_attr_roc_auc:.4f}")

ROC AUC for ATTATTRIBUTE / ATTATTRIBUTE_SIM / AVGATT / RANDOM
0.9305, 0.9471, 0.8927, 0.5043


In [4]:
import torch
import networkx as nx
from torch_geometric.utils import to_networkx, k_hop_subgraph, remove_self_loops, add_self_loops
from torch_geometric.data import Data

dataset_name = 'Infection_50003d_sp'
model_name = f'GAT_infection_3L1H_sp'

# Load the data
data = torch.load(f'/workspace/{dataset_name}.pt',map_location ='cpu')
# Load the model
model = torch.load(f'/workspace/{model_name}.pt',map_location ='cpu')
model.eval()
# Get the attention weights
with torch.no_grad():
    out = model(data.x, data.edge_index, return_att=True)
    att = model.att 
    # att = average_attention_heads(att)
    # model.att = att

In [5]:
def return_all_ground_truth_edges(unique_solution_explanations):
    edge_mask = torch.zeros_like(data.edge_index[0], dtype=torch.bool)
    all_ground_truth_edges = set()
    for expl_path in data.unique_solution_explanations:
        for curr_idx in range(len(expl_path)-1):
            all_ground_truth_edges.add((expl_path[curr_idx], expl_path[curr_idx+1]))

    for idx, curr_edge in enumerate(data.edge_index.t()):
        if (curr_edge[0].item(), curr_edge[1].item()) in all_ground_truth_edges:
            edge_mask[idx] = True

    return edge_mask

In [6]:
data.edge_mask = return_all_ground_truth_edges(data.unique_solution_explanations)

In [35]:
from tqdm import tqdm
from torch_geometric.explain import Explainer, GNNExplainer

explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=50),
    explanation_type='phenomenon',
    node_mask_type='object',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='raw',
    ),
)



In [37]:
from sklearn.metrics import roc_auc_score
# Explanation ROC AUC over all test nodes:
targets, preds = [], []
node_indices = range(300, 700)
for node_index in tqdm(node_indices, leave=False, desc='Train Explainer'):
    target = data.y
    explanation = explainer(data.x, data.edge_index, index=node_index,
                            target=target)

    _, _, _, hard_edge_mask = k_hop_subgraph(node_index, num_hops=3,
                                            edge_index=data.edge_index)

    targets.append(data.edge_mask[hard_edge_mask].cpu())
    preds.append(explanation.edge_mask[hard_edge_mask].cpu())

auc = roc_auc_score(torch.cat(targets), torch.cat(preds))
print(f'Mean ROC AUC (explanation type phenomenon): {auc:.4f}')

Mean ROC AUC (explanation type phenomenon): 0.4972


In [39]:
auc

0.4971986745517879

In [11]:
from tqdm import tqdm
from torch_geometric.explain import Explainer, PGExplainer

explainer = Explainer(
    model=model,
    algorithm=PGExplainer(epochs=30, lr=0.003),
    explanation_type='phenomenon',
    # node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='raw',
    ),
)

for epoch in range(30):
    for index in range(300, 700):  # Indices to train against.
        loss = explainer.algorithm.train(epoch, model, data.x, data.edge_index,
                                        target=target, index=index)
        
# Explanation ROC AUC over all test nodes:
targets, preds = [], []
node_indices = range(300, 700)
for node_index in tqdm(node_indices, leave=False, desc='Train Explainer'):
    target = data.y
    explanation = explainer(data.x, data.edge_index, index=node_index,
                            target=target)

    _, _, _, hard_edge_mask = k_hop_subgraph(node_index, num_hops=3,
                                            edge_index=data.edge_index)

    targets.append(data.edge_mask[hard_edge_mask].cpu())
    preds.append(explanation.edge_mask[hard_edge_mask].cpu())

auc_pgexpl = roc_auc_score(torch.cat(targets), torch.cat(preds))
print(f'Mean ROC AUC (explanation type phenomenon): {auc_pgexpl:.4f}')

Mean ROC AUC (explanation type phenomenon): 0.7198


In [12]:
auc_pgexpl

0.7197579851612776